In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
import tensorflow_datasets as tfds
ds, info = tfds.load('caltech_birds2011', split='train', shuffle_files=True, with_info=True)

In [ ]:
print("Num classes: " + str(info.features['label'].num_classes))
print("Class names: " + str(info.features['label'].names))

In [ ]:
vis = tfds.visualization.show_examples(ds, info, image_key='image')

In [ ]:
del ds
del info

In [ ]:
batch_size = 64
IMG_SIZE = 224 

dataset_name = "caltech_birds2011"
(ds_train, ds_test, ds_val), ds_info = tfds.load(
    dataset_name, split=['train+test[:3428]', 'test[3428:4611]', 'test[4611:]'], with_info=True, as_supervised=True
) # 80 - 10 - 10 split
NUM_CLASSES = ds_info.features["label"].num_classes

In [ ]:
import tensorflow as tf
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))
ds_val = ds_val.map(lambda image, label: (tf.image.resize(image, size), label))

In [ ]:
import matplotlib.pyplot as plt

label_info = ds_info.features["label"]
for i, (image, label) in enumerate(ds_train.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image.numpy().astype("uint8"))
    # plt.title("{}".format(label))
    plt.axis("off")

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

img_augmentation = Sequential(
    [
        preprocessing.RandomRotation(factor=0.15),
        preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
        preprocessing.RandomFlip(),
        preprocessing.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [ ]:
for image, label in ds_train.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        aug_img = img_augmentation(tf.expand_dims(image, axis=0))
        plt.imshow(aug_img[0].numpy().astype("uint8"))
        plt.title("{}".format(label))
        plt.axis("off")

In [ ]:
# One-hot / categorical encoding
def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

ds_val = ds_val.map(input_preprocess)
ds_val = ds_val.batch(batch_size=batch_size, drop_remainder=True)

### EfficientNet Model

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB7

def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB7(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
model = build_model(num_classes=NUM_CLASSES)
epochs = 50
hist = model.fit(ds_train, epochs=epochs, validation_data=ds_val, verbose=1)

In [ ]:
model.summary()

In [ ]:
result = model.evaluate(ds_test, batch_size=128)

In [ ]:
print("EfficientNetB7 Test:")
print(f"Accuracy: {result[1]*100:.3f}%")
print(f"Loss: {result[0]:.3f}")

In [ ]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


plot_hist(hist)

In [ ]:
import os
import shutil
import sys
import random
import errno
import numpy as np
import keras
from keras import models, layers, optimizers
from keras.applications import vgg16, resnet50
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import decode_predictions
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Activation,Concatenate
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def load_image_paths(dataset_path, path_prefix=''):
    paths = {}

    with open(os.path.join(dataset_path, 'images.txt')) as f:
        for line in f:
            (k, p) = line.split()
            path = os.path.join(path_prefix, p)
            paths[int(k)] = path

    return paths

def load_class_names(dataset_path):
    classes = {}

    with open(os.path.join(dataset_path, "classes.txt")) as f:
        for line in f:
            (k, c) = line.split()
            classes[int(k)] = c

    return classes

def load_image_labels(dataset_path):
    labels = {}

    with open(os.path.join(dataset_path, "image_class_labels.txt")) as f:
        for line in f:
            (k, c) = line.split()
            labels[int(k)] = int(c)

    return labels

def split_each_class(class_names, image_labels, split_train=0.60, split_val=0.20, split_test=0.20):
    splits = {}

    for c in class_names.keys():
        # Find all images with label c
        class_images = [k for k,v in image_labels.items() if v == c]

        # Count images with label c
        class_count = len(class_images)

        # Split 60/20/20 train/val/test
        train_count = round(class_count * split_train)
        val_count = round(class_count * split_val)
        test_count = round(class_count * split_test)

        image_indices = list(range(class_count))
        random.shuffle(image_indices)

        train_indices = image_indices[0:train_count]
        val_indices = image_indices[train_count:train_count+val_count]
        test_indices = image_indices[train_count+val_count:]

        for i in train_indices:
            splits[class_images[i]] = 0
        for i in val_indices:
            splits[class_images[i]] = 1
        for i in test_indices:
            splits[class_images[i]] = 2

    return splits

def copy_by_split(class_splits, image_paths, source_base, destination_base):
    folders = {0: "train", 1: "val", 2: "test"}

    for k,v in class_splits.items():
        old_path = os.path.join(source_base, image_paths[k])
        new_path = os.path.join(destination_base, folders[v], image_paths[k])
        try:
            shutil.copy2(old_path, new_path)
        except IOError as e:
            if e.errno != errno.ENOENT:
                raise
            os.makedirs(os.path.dirname(new_path))
            shutil.copy2(old_path, new_path)


In [ ]:
dataset_path = "/Users/pariv/OneDrive/Documents/FA20 - INFO-H518-33436 Deep Learning Neural Network/Final Project/datafile/CUB_200_2011"
image_path_prefix = "images"
destination_path = "/Users/pariv/OneDrive/Documents/FA20 - INFO-H518-33436 Deep Learning Neural Network/Final Project/data/CUB_200_2011"

In [ ]:
class_names = load_class_names(dataset_path)
image_labels = load_image_labels(dataset_path)
image_paths = load_image_paths(dataset_path, image_path_prefix)

In [ ]:
class_splits = split_each_class(class_names, image_labels, 0.60, 0.20, 0.20)
copy_by_split(class_splits, image_paths, dataset_path, destination_path)

In [ ]:
train_dir = "/content/drive/My Drive/data/CUB_200_2011/train/images"
val_dir = "/content/drive/My Drive/data/CUB_200_2011/val/images"
test_dir = "/content/drive/My Drive/data/CUB_200_2011/test/images"

classes_count = 200
image_size = 224
history = None

### VGG16 Model

In [ ]:
vgg_model = vgg16.VGG16(weights="imagenet")
base_model = vgg16.VGG16
trainable_layers = 4

base_model = base_model(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))

for layer in base_model.layers[:-trainable_layers]:
    layer.trainable = False

bird_model = models.Sequential()
bird_model.add(base_model)
bird_model.add(layers.Flatten())
bird_model.add(layers.Dense(1024, activation="relu"))
bird_model.add(layers.Dropout(0.5))
bird_model.add(layers.Dense(classes_count, activation="softmax"))

In [ ]:
# model summary
bird_model.summary()

In [ ]:
#generate data
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,height_shift_range=0.2,
                                   horizontal_flip=True, fill_mode="nearest")
validation_datagen = ImageDataGenerator(rescale=1./255)
train_batchsize = 100
validation_batchsize = 10
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(image_size, image_size),
                                                    batch_size=train_batchsize, class_mode="categorical")
validation_generator = validation_datagen.flow_from_directory(val_dir, target_size=(image_size, image_size),
                                                              batch_size=validation_batchsize, class_mode="categorical",
                                                              shuffle=False)

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=10, verbose=0, mode="auto")

# Compiling the model
bird_model.compile(loss="categorical_crossentropy", optimizer=optimizers.RMSprop(lr = 1e-4), metrics=["accuracy"])

history = bird_model.fit(train_generator, callbacks=[early_stop], 
                         steps_per_epoch=train_generator.samples/train_generator.batch_size, epochs=100,
                         validation_data=validation_generator, 
                         validation_steps=validation_generator.samples/validation_batchsize, verbose=1)

# Saving model
bird_model.save("bird_model_vgg16_Cub_200_2011.h5")

In [ ]:
# learning curves of the training and validation accuracy/loss of VGG-16 model
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('VGG16 Model Training and Validation Accuracy')

plt.xlabel('epoch')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('VGG16 Model Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('VGG16 Model Training and Validation Accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
#Test VGG-16 model
bird_model = models.load_model("bird_model_vgg16_Cub_200_2011.h5")

bird_model.compile(loss="categorical_crossentropy", optimizer=optimizers.RMSprop(lr=1e-4), metrics=["accuracy"])
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_batchsize = 10
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(image_size, image_size),
                                                  batch_size=test_batchsize, class_mode="categorical")

history = bird_model.evaluate(test_generator,steps=test_generator.samples / test_generator.batch_size, verbose=1)
print(history)

In [ ]:
#VGG16 model test scores
print("VGG16 Test")
print("Accuracy:", history[1])
print("Loss:",history[0])


### ResNet50 Model

In [ ]:
resnet_model = resnet50.ResNet50(weights="imagenet")
base_model = resnet50.ResNet50
base_model = base_model(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))

bird_model = models.Sequential()
bird_model.add(base_model)
bird_model.add(layers.Flatten())
bird_model.add(layers.Dense(1024, activation="relu"))
bird_model.add(layers.Dropout(0.5))
bird_model.add(layers.Dense(classes_count, activation="softmax"))

In [ ]:
# model summary
bird_model.summary()

In [ ]:
#generate data
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,height_shift_range=0.2,
                                   horizontal_flip=True, fill_mode="nearest")
validation_datagen = ImageDataGenerator(rescale=1./255)
train_batchsize = 100
validation_batchsize = 10
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(image_size, image_size),
                                                    batch_size=train_batchsize, class_mode="categorical")
validation_generator = validation_datagen.flow_from_directory(val_dir, target_size=(image_size, image_size),
                                                              batch_size=validation_batchsize, class_mode="categorical",
                                                              shuffle=False)

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=10, verbose=0, mode="auto")

# Compiling the model
bird_model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=1e-4), metrics=["accuracy"])

history = bird_model.fit(train_generator, callbacks=[early_stop],
                         steps_per_epoch=train_generator.samples/train_generator.batch_size, epochs=23,
                         validation_data=validation_generator,
                         validation_steps=validation_generator.samples/validation_batchsize, verbose=1)

# Saving model
bird_model.save("bird_model_resnet50.h5")

In [ ]:
# learning curves of the training and validation accuracy/loss of ResNet50 model
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('ResNet50 Model Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('ResNet50 Model Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('ResNet50 Model Training and Validation Accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
#test the model
bird_model = models.load_model("bird_model_resnet50.h5")
bird_model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=1e-4), metrics=["accuracy"])
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_batchsize = 10
test_generator = test_datagen.flow_from_directory(test_dir,target_size=(image_size, image_size), 
                                                  batch_size=test_batchsize, class_mode="categorical")

history = bird_model.evaluate(test_generator, steps=test_generator.samples / test_generator.batch_size, verbose=1)
print(history)

In [ ]:
#model test scores
print("ResNet50 Test")
print("Accuracy:",history[1])
print("Loss:",history[0])


In [ ]:
dir_data_img = "/content/drive/My Drive/CUB_200_2011/images"
dir_bird_file = "017.Cardinal/Cardinal_0014_17389"
path_to_bird_img = os.path.join(dir_data_img, dir_bird_file + ".jpg")

In [ ]:
bird_original = load_img(path_to_bird_img, target_size=(224, 224))
bird_numpy = img_to_array(bird_original)
plt.imshow(np.uint8(bird_numpy))
bird_batch = np.expand_dims(bird_numpy, axis=0)
bird_processed = vgg16.preprocess_input(bird_batch.copy())

predictions = resnet_model.predict(bird_processed)
label = decode_predictions(predictions)
print(label)

### CNN Model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adamax
from keras import models, layers, optimizers
from keras.applications import vgg16, resnet50
from keras.applications.imagenet_utils import decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img

import os 
import shutil
import sys
import random
import errno


In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
classes_count = 200
image_size = 224
input_shape = (224, 224, 3)
history = None

In [ ]:
# Model Build
model = Sequential()
model.add(Conv2D(32, kernel_size=(5,5),padding='same', input_shape=input_shape,activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3),padding='same', activation ='relu'))                                                                   
model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2)))
model.add(Conv2D(96, kernel_size=(3,3),padding='same', activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2)))
model.add(Conv2D(96, kernel_size=(3,3),padding='same', activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2)))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(200, activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:
#generate data
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,height_shift_range=0.2,
                                   horizontal_flip=True, fill_mode="nearest")
validation_datagen = ImageDataGenerator(rescale=1./255)
train_batchsize = 100
validation_batchsize = 10
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(image_size, image_size),
                                                    batch_size=train_batchsize, class_mode="categorical")
validation_generator = validation_datagen.flow_from_directory(val_dir, target_size=(image_size, image_size),
                                                              batch_size=validation_batchsize, class_mode="categorical",
                                                              shuffle=False)

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=10, verbose=0, mode="auto")

model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(train_generator, callbacks=[early_stop], 
                    steps_per_epoch=train_generator.samples/train_generator.batch_size, epochs=100,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples/validation_batchsize, verbose=1)
model.save("bird_model_CNN_cub-200-2011.h5")

In [ ]:
def plotloss(history,both = True):
    plt.plot(history.history['loss'])
    if both:
        plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    if both:
        plt.legend(['train', 'validation'])
    plt.show()

In [ ]:
# learning curves of the training and validation accuracy/loss of CNN model
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('CNN Model Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='lower left')
plt.ylabel('Cross Entropy')
plt.title('CNN Model Training and Validation Loss')
plt.xlabel('epoch')
plt.show()